In [2]:
!pip install pulp

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 14.3/14.3 MB 1.8 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [9]:
import pulp

#initialize problem class
Lp_prob=pulp.LpProblem("Cost minimizing blending problem",pulp.LpMinimize)

#construct our decision variable lists
sausage_types=['economy','premium']
ingredients=['pork','wheat','starch']

#declare variables
ing_weight=pulp.LpVariable.dicts("weight kg",
                                ((i,j)for i in sausage_types for j in ingredients),
                                 lowBound=0,
                                 cat="Continuous")
print("Variables \n",ing_weight)

#objective function
Lp_prob += (pulp.lpSum([4.32 * ing_weight[(i,'pork')]+ 2.46 * ing_weight[(i,'wheat')] 
                       + 1.86 * ing_weight[(i,'starch')]for i in sausage_types]))

#constraints
#350 economy and 500 premium sausages at 0.05 kg
Lp_prob += pulp.lpSum([ing_weight['economy',j]for j in ingredients])==350 * 0.05
Lp_prob += pulp.lpSum([ing_weight['premium',j]for j in ingredients])==500 * 0.05

#economy has >= 40% pork,premium >= 60% pork
Lp_prob += ing_weight['economy','pork']>=(0.4 * pulp.lpSum([ing_weight['economy',j]for j in ingredients]))
Lp_prob += ing_weight['premium','pork']>=(0.6 * pulp.lpSum([ing_weight['premium',j]for j in ingredients]))

#sausages must be <= 25% starch
Lp_prob += ing_weight['economy','starch']<=(0.25 * pulp.lpSum([ing_weight['economy',j]for j in ingredients]))
Lp_prob += ing_weight['premium','starch']<=(0.25 * pulp.lpSum([ing_weight['premium',j]for j in ingredients]))

#we have at most 30kg of pork,20kg of wheat and 17kg of starch available
Lp_prob += pulp.lpSum([ing_weight[i,'pork']for i in sausage_types])<=30
Lp_prob += pulp.lpSum([ing_weight[i,'wheat']for i in sausage_types])<=20
Lp_prob += pulp.lpSum([ing_weight[i,'starch']for i in sausage_types])<=17

#we have at least 23kg of pork to use up
Lp_prob += pulp.lpSum([ing_weight[i,'pork']for i in sausage_types])>=23

#solve the problem
status=Lp_prob.solve()

#display
print("\n Displaying the problem \n..........................\n",Lp_prob)
print("\n Solution status \n..........................\n",pulp.LpStatus[status]) #solution status
print("\n Final solution \n..........................\n")

for var in ing_weight:
    var_value = ing_weight[var].varValue
    print("The weight of",var[1],"in",var[0],"sausages is",var_value,"kg")

total_cost=pulp.value(Lp_prob.objective)

print("The total cost is ${} for 350 economy sausages and 500 premium sausages",format(round(total_cost,2)))



Variables 
 {('economy', 'pork'): weight_kg_('economy',_'pork'), ('economy', 'wheat'): weight_kg_('economy',_'wheat'), ('economy', 'starch'): weight_kg_('economy',_'starch'), ('premium', 'pork'): weight_kg_('premium',_'pork'), ('premium', 'wheat'): weight_kg_('premium',_'wheat'), ('premium', 'starch'): weight_kg_('premium',_'starch')}

 Displaying the problem 
..........................
 Cost_minimizing_blending_problem:
MINIMIZE
4.32*weight_kg_('economy',_'pork') + 1.86*weight_kg_('economy',_'starch') + 2.46*weight_kg_('economy',_'wheat') + 4.32*weight_kg_('premium',_'pork') + 1.86*weight_kg_('premium',_'starch') + 2.46*weight_kg_('premium',_'wheat') + 0.0
SUBJECT TO
_C1: weight_kg_('economy',_'pork') + weight_kg_('economy',_'starch')
 + weight_kg_('economy',_'wheat') = 17.5

_C2: weight_kg_('premium',_'pork') + weight_kg_('premium',_'starch')
 + weight_kg_('premium',_'wheat') = 25

_C3: 0.6 weight_kg_('economy',_'pork') - 0.4 weight_kg_('economy',_'starch')
 - 0.4 weight_kg_('economy